In [1]:
import DataLoading
from torch.utils.data import DataLoader,TensorDataset
import torch
import pandas as pd
import NN_classes
import training_methods
import time

In [2]:
sc = "sc01"
period = "2030"
folder = "../Data/RTS24_AC_12w_ext_o_dummy"
all_executions = DataLoading.list_executions(folder="../Data/RTS24_AC_12w_ext_o_dummy",per = period,sc=sc)
len(all_executions)
executions_start = 0 
executions_end = len(all_executions)
executions = all_executions[executions_start:executions_end]
te_s = 0.3
val_s = 0.4

In [3]:
dfs_in,dfs_out,dfs_inter = DataLoading.load_data_ext_out(folder,executions,period,sc,["PowerOutput"])
dfs_inter_j = DataLoading.join_frames_inter_layer(dfs_inter)
dfs_inter_j = DataLoading.trim_columns_to_common(dfs_inter_j)
ts_in,ts_out,ts_inter = DataLoading.split_tr_val_te_ext_out(dfs_in,dfs_out,dfs_inter_j,executions,te_s,val_s)
d_ft_in, d_ft_out,d_ft_inter = DataLoading.concat_and_normalize_ext_out(ts_in,ts_out,ts_inter,executions)

train = TensorDataset(d_ft_in['train'].float(), d_ft_out['train'].float(),d_ft_inter['train'])
validation = TensorDataset(d_ft_in['val'].float(), d_ft_out['val'].float(),d_ft_inter['val'].float())


input_f_sc01_Network_Existing_Generation_Full_2030.csv
86
input_f_sc01_Network_Line_In_N_101_N_102_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_101_N_103_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_101_N_105_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_102_N_104_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_102_N_106_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_103_N_109_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_103_N_124_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_104_N_109_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_105_N_110_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_106_N_108_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_106_N_110_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_107_N_108_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_108_N_109_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_108_N_110_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_109_N_111_cac1_2030.csv
86
input_f_sc01_Network_Line_In_N_109_N_112_cac1_2030.csv
86
input_f_sc01_N

In [4]:
i=0
nbs_hidden = [(3,2)]
dors = [0]#,0.05,0.1]#,0.05]
relu_outs=[False]

batch_sizes = [32,64,128]
learning_rates = [0.0025*4**i for i in range(-1,3,1)]
nbs_e = [4]#,8,12,16,20] # ,8]
negative_penalisations = [0]
alphas = [0]#,0.1,0.2,0.4]
beta = 1


results = pd.DataFrame()
folder_to_save = "RTS24_AC_12w_ext_o_dummy"

hp_sets = ((nb_h,dor,relu_out,bs,lr,nb_e,np,alpha) for nb_h in nbs_hidden for dor in dors for relu_out in relu_outs for bs in batch_sizes for lr in learning_rates for nb_e in nbs_e for np in negative_penalisations for alpha in alphas)

inter_size = dfs_inter_j["Network_Existing_Generation_Full"].shape[1]

for hp_set in hp_sets:
    print(hp_set)
    nb_hidden,dor,relu_out,bs,lr,nb_e,np,alpha = hp_set[0],hp_set[1],hp_set[2],hp_set[3],hp_set[4],hp_set[5],hp_set[6],hp_set[7]
    
    #Create training and validation loaders based on batch size
    training_loader = DataLoader(train,batch_size=bs)
    validation_loader = DataLoader(validation,batch_size=bs)
    
    #Initialize loss functions
    loss_fn = NN_classes.create_custom_loss(alpha=alpha,beta=beta)
    loss_t_mse = torch.nn.MSELoss()
    
    #Create model based on hyperparameter set
    m = NN_classes.create_model(nb_hidden,d_ft_in['train'].shape[1],dropout_ratio= dor,relu_out=relu_out,inter = True,inter_size=inter_size)
    #Create model name for saving and loading
    m_name = f"OE_{nb_hidden}h_{nb_e}e_{lr}lr_{dor}dor_{np}np_{relu_out}_ro_{bs}bs"
    #Create optimizer based on learning rate 
    optimizer = torch.optim.Adam(m.parameters(), lr=lr)
    #Train the actual model 
    t_start_train = time.perf_counter()
    train_loss_1 = training_methods.train_multiple_epochs(nb_e,m,training_loader,validation_loader,loss_fn,optimizer,m_name,folder_to_save,True)[0]
    t_stop_train = time.perf_counter()
    
    #In the following loop, we retreive the models from saved locations and calculate losses 
    for mt in ["min_val","all_epochs"]:
        t_start_eval = time.perf_counter()
        path = f"trained_models/{folder_to_save}/{mt}/model_{m_name}.pth"
        
        #Retreive model state and set to evaluation mode
        m.load_state_dict(torch.load(path))
        m.eval()
        
        #Calculate losses
        test_predictions = m(d_ft_in["test"].float())
        test_loss = loss_fn(test_predictions[0].squeeze(),d_ft_out["test"],test_predictions[1].squeeze(),d_ft_inter["test"])
        test_loss_t_mse = loss_t_mse(test_predictions[0].squeeze(),d_ft_out["test"])


        train_predictions = m(d_ft_in["train"].float())
        train_loss = loss_fn(train_predictions[0].squeeze(),d_ft_out["train"],train_predictions[1].squeeze(),d_ft_inter["train"])
        train_loss_t_mse = loss_t_mse(train_predictions[0].squeeze(),d_ft_out["train"])


        validation_prediction = m(d_ft_in["val"].float())
        validation_loss = loss_fn(validation_prediction[0].squeeze(),d_ft_out["val"],validation_prediction[1].squeeze(),d_ft_inter["val"])
        validation_loss_t_mse = loss_t_mse(validation_prediction[0].squeeze(),d_ft_out["val"])
        t_stop_eval = time.perf_counter()
        
        
        #Calculate some calculation times 
        t_train = t_stop_train - t_start_train
        t_eval = t_stop_eval - t_start_eval
        
        #Finally, save all desired values in a dataframe
        r = pd.DataFrame({"Model_type": [nb_hidden],
                        "Dor": dor,
                        "Relu_out": relu_out,
                        "Batch_size": bs,
                        "Lr":lr,
                        "Epochs": nb_e,
                        "Np": np,
                        "Min_val":mt,
                        "Tr_l":train_loss.item(),
                        "Te_l":test_loss.item(),
                        "V_l": validation_loss.item(),
                        "Tr_l_t_mse":train_loss_t_mse.item(),
                        "Te_l_t_mse":test_loss_t_mse.item(),
                        "V_l_t_mse": validation_loss_t_mse.item(),
                        "Tr_l_ret": train_loss_1.item(),
                        "Train_time": t_train,
                        "Eval_time": t_eval,
                        "alpha": alpha, 
                        "beta": beta,
                        "Test size": te_s,
                        "Val size": val_s
                         }
                        ,index = [i])
        i+=1
        results = pd.concat([results,r])



((3, 2), 0, False, 32, 0.000625, 4, 0, 0)
[172, 258, 68, 51, 25, 12]
EPOCH 1:
LOSS train 0.0003701292724234611 valid 9.172188583761454e-06
EPOCH 2:
LOSS train 1.4808920789863156e-06 valid 7.650974112038966e-06
EPOCH 3:
LOSS train 1.216020439435784e-06 valid 7.745312359475065e-06
EPOCH 4:
LOSS train 1.5887459702385783e-06 valid 8.677906407683622e-06
((3, 2), 0, False, 32, 0.0025, 4, 0, 0)
[172, 258, 68, 51, 25, 12]
EPOCH 1:
LOSS train 0.0006028931329370164 valid 1.9033628632314503e-05
EPOCH 2:
LOSS train 3.705200253493011e-06 valid 1.6362111637135968e-05
EPOCH 3:
LOSS train 2.6335840234384097e-06 valid 1.2521814824140165e-05
EPOCH 4:
LOSS train 1.8495277298574431e-06 valid 9.322238838649355e-06
((3, 2), 0, False, 32, 0.01, 4, 0, 0)
[172, 258, 68, 51, 25, 12]
EPOCH 1:
LOSS train 0.0001457202085643404 valid 8.641240856377408e-05
EPOCH 2:
LOSS train 1.7764181435182868e-05 valid 9.330770262749866e-05
EPOCH 3:
LOSS train 1.9744108456260855e-05 valid 9.44960120250471e-05
EPOCH 4:
LOSS train 1

KeyboardInterrupt: 

In [33]:
results.to_csv("Loss_results_csv/All_Exec_inter_dummy_test.csv")